In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/events.csv', low_memory=False)

In [3]:
def asignar_empresa(celular):
    celular = celular.lower()
    if ('samsung' in celular):
        return 'Samsung'
    if ('iphone' in celular or 'ipad' in celular):
        return 'Apple'
    if ('sony' in celular):
        return 'Sony'
    if ('motorola' in celular):
        return 'Motorola'
    if ('asus' in celular):
        return 'Asus'
    if ('lg' in celular):
        return 'LG'
    if ('lenovo' in celular):
        return 'Lenovo'
    if ('quantum' in celular):
        return 'Quantum'
    return 'Otros'

In [4]:
modelos_por_empresa = df.loc[df['sku'].notnull(), :].dropna(axis=1).copy()
modelos_por_empresa['empresa'] = modelos_por_empresa['model'].apply(lambda x: asignar_empresa(x))

## Marca más vista por usuario

In [5]:
vistos = modelos_por_empresa.loc[modelos_por_empresa['event'] == 'viewed product']

In [6]:
vistos = vistos.groupby('person')['empresa'].value_counts()
vistos = vistos.reset_index(level='empresa', name='count')
vistos = vistos[~vistos.index.duplicated(keep='first')][['empresa']]

In [7]:
vistos = pd.get_dummies(vistos['empresa'], prefix='vio_mas', dtype=bool)

In [8]:
vistos.tail()

,vio_mas_Apple,vio_mas_Asus,vio_mas_LG,vio_mas_Lenovo,vio_mas_Motorola,vio_mas_Quantum,vio_mas_Samsung,vio_mas_Sony
person,,,,,,,,
fff1caee,False,False,True,False,False,False,False,False
fff2bdde,False,False,False,False,False,False,True,False
fff72025,False,False,True,False,False,False,False,False
fff78145,True,False,False,False,False,False,False,False
fffd1246,True,False,False,False,False,False,False,False


In [9]:
users = pd.read_csv('../data/features_basicas.csv', low_memory=False, index_col='person')
users.shape

(38829, 16)

In [10]:
usuarios_con_productos_vistos = users.join(vistos)

In [11]:
usuarios_con_productos_vistos = usuarios_con_productos_vistos.fillna({'vio_mas_Apple':False, 'vio_mas_Samsung':False, 'vio_mas_Lenovo':False,\
                      'vio_mas_LG':False, 'vio_mas_Asus':False, 'vio_mas_Motorola':False,\
                      'vio_mas_Quantum':False, 'vio_mas_Sony':False})
usuarios_con_productos_vistos.columns

Index(['tiene_checkouts', 'llegaron_por_ad', 'llegaron_por_search',
       'tiene_conversions', 'cant_conversions', 'cant_checkouts',
       'cant_viewed_product', 'cant_searched_product', 'cant_visitas',
       'cant_leads', 'cant_brand_listings', 'cant_generic_listings',
       'cant_entradas_ads', 'cant_entradas_buscador', 'total_sesiones',
       'promedio_eventos_por_sesion', 'vio_mas_Apple', 'vio_mas_Asus',
       'vio_mas_LG', 'vio_mas_Lenovo', 'vio_mas_Motorola', 'vio_mas_Quantum',
       'vio_mas_Samsung', 'vio_mas_Sony'],
      dtype='object')

In [12]:
#Descomentar para guardar, un horror
usuarios_con_productos_vistos[['vio_mas_Apple',
 'vio_mas_Asus',
 'vio_mas_LG',
 'vio_mas_Lenovo',
 'vio_mas_Motorola',
 'vio_mas_Quantum',
 'vio_mas_Samsung',
 'vio_mas_Sony']].to_csv('../data/features_vistos.csv', sep=',')

## Checkouts según marca

In [13]:
checkouts = modelos_por_empresa.loc[modelos_por_empresa['event'] == 'checkout']

In [14]:
checkouts_agrupadas = checkouts.groupby('person')['empresa'].value_counts().unstack(fill_value=0)

In [15]:
checkouts_agrupadas = checkouts_agrupadas.apply(lambda x: x > 0, axis=1)
checkouts_agrupadas = checkouts_agrupadas.add_prefix('checkout_')
checkouts_agrupadas.shape

(32833, 8)

In [16]:
users_con_checkouts = users.join(checkouts_agrupadas)

In [17]:
users_con_checkouts.shape

(38829, 24)

In [18]:
users_con_checkouts.fillna(False, inplace=True)

In [19]:
users_con_checkouts.columns

Index(['tiene_checkouts', 'llegaron_por_ad', 'llegaron_por_search',
       'tiene_conversions', 'cant_conversions', 'cant_checkouts',
       'cant_viewed_product', 'cant_searched_product', 'cant_visitas',
       'cant_leads', 'cant_brand_listings', 'cant_generic_listings',
       'cant_entradas_ads', 'cant_entradas_buscador', 'total_sesiones',
       'promedio_eventos_por_sesion', 'checkout_Apple', 'checkout_Asus',
       'checkout_LG', 'checkout_Lenovo', 'checkout_Motorola',
       'checkout_Quantum', 'checkout_Samsung', 'checkout_Sony'],
      dtype='object')

In [20]:
# Descomentar para guardar, un horror
users_con_checkouts[['checkout_Apple', 'checkout_Asus', 'checkout_LG', 'checkout_Lenovo',
       'checkout_Motorola', 'checkout_Quantum', 'checkout_Samsung', 'checkout_Sony']].to_csv('../data/features_checkouts.csv')